In [1]:
from Model.model import ModelSTGCN
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pds
import numpy as np
import torch.nn as nn
import copy

c:\Users\ADMIN\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ActionDataset(Dataset):
    def __init__(self,Transform=None):
        super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        self.transform = Transform
        self.lst=[]
        self.label=[]
        # self.append(Data,label)

    def __getitem__(self, index):
        sample = self.lst[index],self.label[index]
        if self.transform:
            sample=self.transform(sample)
        return sample
    
    def __len__(self):
        return self.leng
    
    def SetTrans(self,Transform):
        self.transform=Transform
    
    def append(self,Data):
        # super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        # assert label==None,"Label must be give"
        kpscore = np.expand_dims(Data['keypoint_score'][0],axis=2)
        kp = Data['keypoint'][0]
        shapeimg=Data['img_shape']
        ## normalize pic
        kp[...,0]=kp[...,0]/shapeimg[1]
        kp[...,1]=kp[...,1]/shapeimg[0]
        #############
        data = np.concatenate((kp,kpscore),axis=2)
        # data = np.expand_dims(data,axis=0)
        label = Data['label']
        self.lst.append(data)
        self.label.append(label)
        self.leng=len(self.lst)  

class ToTensor():
    def __call__(self,sample):
        data,label=sample
        return torch.from_numpy(data.astype(np.float32)),torch.tensor(label)

In [3]:
# a=torch.rand(2,1).argmax(dim=0)
# val =a.item()

In [4]:
# torch.__version__

In [5]:
def Train_epoch(model=None,loss_fn=None,train_loader=None,optimizer=None,device='cuda'):
    model.train()
    # model.training = True
    total_loss=0
    for index,(data,label) in enumerate(train_loader):
        outputs=model(data.to(device))
        label=label.to(device)
        loss = loss_fn(outputs,label)
        total_loss+=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    total_loss/=(index+1)
    return total_loss    

def Val_epoch(model=None,loss_fn=None,val_loader=None,device='cuda'):
    device=torch.device(device if torch.cuda.is_available() else 'cpu')
    model.eval()
    model.to(device)
    total_loss = 0
    preds = []
    labels = []
    model.training = True
    with torch.no_grad():
        for index,(data,label) in enumerate(val_loader):
            outputs = model(data.to(device))
            label = label.to(device)
            loss = loss_fn(outputs,label)
            pred = torch.argmax(outputs,dim=1)
            labels.append(label.item())
            preds.append(pred.item())
            total_loss+=loss
    total_loss/=(index + 1)
    #acc multi class CrossEntropy
    acc = eval_acc(preds,labels)
    return total_loss,acc

def eval_acc(preds,labels):
    n_total = len(preds)
    n_correct = 0
    for pred,label in zip(preds,labels):
        if pred == label: n_correct+=1
        else: continue
    acc=n_correct/n_total
    return acc
        

In [10]:
model_best_wts = None
def Train_n_Eval(model:nn.Module=None,epochs=None,loss_fn=None,lr=1e-4,optim=None,
                 train_dataloader=None,eval_dataloader=None,lr_shedule=False,
                 Step=10,miles=2,Gamma=0.1,device='cuda'):
    assert all(param is not None for param in [model,epochs,loss_fn,optim,
                                               train_dataloader,eval_dataloader]),"All Param must be give in"
    device=torch.device(device if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optim = optim(model.parameters(),lr)
    loss_history = {
        'train': [],
        'val' : [],
    }
    best_score=0
    if lr_shedule:
        Multistep=[Step * i for i in range(1,miles+1)]
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optim,Multistep,Gamma)
    for epoch in range(1,epochs+1):
        train_loss = Train_epoch(model,loss_fn,train_dataloader,optim)
        val_loss,acc = Val_epoch(model,loss_fn,eval_dataloader)
        loss_history['train'].append(train_loss)
        loss_history['val'].append(val_loss)
        if acc > best_score:
            best_score = acc
            model_best_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(),'Model_best_wts.pt')
            print("Copied best model weights!")
        if lr_shedule:
            scheduler.step()
        lr=optim.param_groups[-1]['lr']
        print(f'Epoch: {epoch}: Learning rate: {lr}\n \tTrain Loss: {train_loss}\n\tVal Loss: {val_loss}, Accuracy: {acc}')
    torch.save(model.state_dict(),'Checkpoint/model_final.pth')
        

In [23]:
model=ModelSTGCN(3,2)
criterion=nn.CrossEntropyLoss()
optim=torch.optim
datalst1 = pds.read_pickle('Data/train.pkl')
datalst2 = pds.read_pickle('Data/val.pkl')
train_dataset = ActionDataset(ToTensor())
val_dataset = ActionDataset(ToTensor())
for data in datalst1:
    train_dataset.append(data)
for data in datalst2:
    val_dataset.append(data)
train_dataloader=DataLoader(dataset=train_dataset,shuffle=True)
val_dataloader=DataLoader(dataset=val_dataset,shuffle=True)

In [24]:
Train_n_Eval(model=model,epochs=10,loss_fn=criterion,optim=optim.Adam,
             train_dataloader=train_dataloader,eval_dataloader=val_dataloader,
             lr=1e-4,lr_shedule=False,Step=3)

Copied best model weights!
Epoch: 1: Learning rate: 0.0001
 	Train Loss: 0.629124641418457
	Val Loss: 0.488568514585495, Accuracy: 0.78
Epoch: 2: Learning rate: 0.0001
 	Train Loss: 0.4662247896194458
	Val Loss: 0.7534850239753723, Accuracy: 0.62
Epoch: 3: Learning rate: 0.0001
 	Train Loss: 0.3653312921524048
	Val Loss: 0.46664607524871826, Accuracy: 0.78
Copied best model weights!
Epoch: 4: Learning rate: 0.0001
 	Train Loss: 0.28156086802482605
	Val Loss: 0.49483776092529297, Accuracy: 0.8
Epoch: 5: Learning rate: 0.0001
 	Train Loss: 0.21545028686523438
	Val Loss: 0.48174163699150085, Accuracy: 0.78
Epoch: 6: Learning rate: 0.0001
 	Train Loss: 0.1971704065799713
	Val Loss: 0.45127221941947937, Accuracy: 0.78
Epoch: 7: Learning rate: 0.0001
 	Train Loss: 0.1525350660085678
	Val Loss: 0.697263777256012, Accuracy: 0.74
Epoch: 8: Learning rate: 0.0001
 	Train Loss: 0.12032556533813477
	Val Loss: 2.011941432952881, Accuracy: 0.6
Epoch: 9: Learning rate: 0.0001
 	Train Loss: 0.102064214

In [29]:
model.load_state_dict(torch.load('Checkpoint/model_final.pth'))
model.eval()
filetest=pds.read_pickle('Data/test.pkl')
testdata=ActionDataset(ToTensor())
for data in filetest:
    testdata.append(data)
test_loader=DataLoader(testdata,shuffle=True)
total_loss = 0
preds = []
labels = []
with torch.no_grad():
    for index,(data,label) in enumerate(test_loader):
        outputs = model(data.to('cuda'))
        label = label.to('cuda')
        loss = criterion(outputs,label)
        pred = torch.argmax(outputs,dim=1)
        labels.append(label.item())
        preds.append(pred.item())
        total_loss+=loss
total_loss/=(index + 1)
#acc multi class CrossEntropy
acc = eval_acc(preds,labels)

In [30]:
torch.cuda.empty_cache()

In [31]:
print(model_best_wts)

None


In [32]:
acc

0.8

In [14]:
torch.save(model.state_dict(),'Model.pth')

In [ ]:
# a=datalst1[0]['keypoint'][0]
# shapeimg=datalst1[0]['img_shape']
# datalst1[0]['keypoint'][0]


In [ ]:
# a[...,0]=a[...,0]/shapeimg[1]
# a[...,1]=a[...,1]/shapeimg[0]
# a

In [ ]:
# def Train_val(model=None,dataloader=None,epochs=None,lr=1e-4,lr_schedule=None):
#     assert all(param is not None for param in [model,dataloader,epochs]),"All param must be give in: model,dataloader,epochs"
#     device=torch.device('cuda')
#     model.to(device)
#     optim=torch.optim.Adam(model.parameters(),lr)
#     criteria=nn.CrossEntropyLoss()
#     # criteria=nn.BCELoss()
#     for epoch in range(epochs):
#         for i,(data,label) in enumerate (dataloader):
#             output=model(data.to(device))
#             label=label.to(device)
#             # _,predict=torch.max(output,dim=1)
#             loss = criteria(output,label)
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#             print(f'Epoch: {epoch}, step: {i}/{len(dataloader)}, Loss: {loss:.5f}')
    

In [ ]:
# datalst1 = pds.read_pickle('Data/train.pkl')
# datalst2 = pds.read_pickle('Data/val.pkl')
# train_dataset = ActionDataset(ToTensor())
# val_dataset = ActionDataset(ToTensor())
# for data in datalst1:
#     train_dataset.append(data)
# for data in datalst2:
#     val_dataset.append(data)


In [ ]:
# # for epoch in range(2):
# #     for i,data in enumerate (dataloader):
# #         print(f'Epoch: {epoch}, Step: {i+1}/{len(dataloader)}, label: {data[1]}')
# dataview=iter(dataloader)
# data = dataview.next()

In [ ]:
# testfile=pds.read_pickle('test.pkl')

In [ ]:
# testact=ActionDataset(ToTensor())
# testact.append(testfile)

In [ ]:
# torch.expand_copy()

In [ ]:
# batch_size=4
# datalst=pds.read_pickle('train.pkl')
# actDataset=ActionDataset(datalst[0])
# dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
# n_sample=len(actDataset)
# epochs=10
# n_iter=round(n_sample/batch_size)
# for epoch in range(epochs-8):
#     for i,data in enumerate (dataloader):
#         if (i+1)% (batch_size)==0:
#             print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}, Input shap: {data.shape}')

In [ ]:
# import math
# math.ceil(len(actDataset)/batch_size)

In [ ]:
# batch_size=4
# datalst=pds.read_pickle('train.pkl')
# actDataset=ActionDataset(datalst[0])
# dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
# n_sample=len(actDataset)
# n_iter=round(n_sample/batch_size)
# model = ModelSTGCN(3,5)
# lr = 1e-3
# optim = torch.optim.Adam(model.parameters(),lr)
# lr_schedule = torch.optim.lr_scheduler.MultiStepLR(optim,[15,30],gamma=0.1)
# epochs = 10
# for epoch in range(epochs):
#     for i,data in enumerate (dataloader):
#         if (i+1)% (batch_size)==0:
#             print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}')